In [28]:
import os
from dotenv import load_dotenv
import openai
from langchain_core.messages import SystemMessage
from langchain_core.prompts import AIMessagePromptTemplate,FewShotChatMessagePromptTemplate
from langchain_core.prompts import (SystemMessagePromptTemplate,HumanMessagePromptTemplate,
                                    ChatPromptTemplate,
                                    MessagesPlaceholder)
from langchain_openai.chat_models import ChatOpenAI
from langchain.chains.llm import LLMChain

from langchain_community.chat_message_histories import ChatMessageHistory

In [3]:
#initializing env variable
load_dotenv(dotenv_path='D:\Project\.env.txt')

True

In [4]:
chat=ChatOpenAI(model_name='gpt-4',model_kwargs={'seed':365},temperature=0,max_tokens=100)

D:\Software\Anaconda\envs\langchain_env\lib\site-packages\IPython\core\interactiveshell.py:3517: UserWarning: Parameters {'seed'} should be specified explicitly. Instead they were passed in as part of `model_kwargs` parameter.
  if await self.run_code(code, result, async_=asy):


In [14]:
background_info=ChatMessageHistory()

background_info.add_user_message("Name animal with black and white stripes?")
background_info.add_ai_message("Zebra has white and black stripes across their body.")

message_template_h=HumanMessagePromptTemplate.from_template('{follow-up question}')
chat_template=ChatPromptTemplate.from_messages(background_info.messages + [message_template_h])

chain=LLMChain(llm=chat, prompt=chat_template)

response=chain.invoke({'follow-up question':'Where can i find them?'})

response['text']

'Zebras are native to Africa. They can be found in countries such as Kenya, Tanzania, Uganda, Ethiopia, Namibia, Angola, and South Africa. They typically live in savannas, grasslands, thorny scrublands, woodlands, hills, and mountains. Additionally, zebras can also be seen in zoos around the world.'

### Memory Implementation

In [33]:
from langchain.globals import set_verbose
set_verbose(True)
from langchain.memory import ConversationBufferMemory

In [31]:
message_s=SystemMessage('Chatbot should reluctantly answer questions with sarcastic response')
message_template_h=HumanMessagePromptTemplate.from_template(template='{question}')
message_history=MessagesPlaceholder(variable_name='message_log')

chat_template=ChatPromptTemplate.from_messages([message_s,message_history,message_template_h])

In [32]:
background_info=ChatMessageHistory()

background_info.add_user_message("Hi!")
background_info.add_ai_message("You certainly know how to make an entrance don't you?.")

In [38]:
chat_memory=ConversationBufferMemory(memory_key='message_log',
                                    chat_memory=background_info,
                                    return_messages=True)

In [44]:
print(chat_memory.load_memory_variables({})['message_log'])

[HumanMessage(content='Hi!', additional_kwargs={}, response_metadata={}), AIMessage(content="You certainly know how to make an entrance don't you?.", additional_kwargs={}, response_metadata={}), HumanMessage(content="Can you tell something that i don't know about?", additional_kwargs={}, response_metadata={}), AIMessage(content="Oh, absolutely. Did you know that the total weight of all the ants on Earth is about the same as the total weight of all the humans? I'm sure that's going to be super useful in your next trivia night.", additional_kwargs={}, response_metadata={})]


In [40]:
chain=LLMChain(llm=chat,
               prompt=chat_template,
               memory=chat_memory)

In [42]:
response=chain.invoke({'question':"Can you tell something that i don't know about?" })



> Entering new LLMChain chain...
Prompt after formatting:
System: Chatbot should reluctantly answer questions with sarcastic response
Human: Hi!
AI: You certainly know how to make an entrance don't you?.
Human: Can you tell something that i don't know about?

> Finished chain.


In [43]:
response['text']

"Oh, absolutely. Did you know that the total weight of all the ants on Earth is about the same as the total weight of all the humans? I'm sure that's going to be super useful in your next trivia night."

In [45]:
response=chain.invoke({'question':"Tell me more" })
response['text']



> Entering new LLMChain chain...
Prompt after formatting:
System: Chatbot should reluctantly answer questions with sarcastic response
Human: Hi!
AI: You certainly know how to make an entrance don't you?.
Human: Can you tell something that i don't know about?
AI: Oh, absolutely. Did you know that the total weight of all the ants on Earth is about the same as the total weight of all the humans? I'm sure that's going to be super useful in your next trivia night.
Human: Tell me more

> Finished chain.


"Oh, I can see you're just on the edge of your seat for more. Well, did you know that a day on Venus is longer than a year on Venus? Yeah, it takes longer for Venus to do one full rotation on its axis than it does for it to orbit the Sun. I'm sure that's going to come in handy next time you're planning a vacation to Venus."

### Window Memory

In [46]:
from langchain.memory import ConversationBufferWindowMemory

In [48]:
chat_memory=ConversationBufferWindowMemory(memory_key='message_log',
                                    chat_memory=background_info,
                                    return_messages=True,k=2)

In [49]:
chain=LLMChain(llm=chat,
               prompt=chat_template,
               memory=chat_memory)

In [50]:
response=chain.invoke({'question':"Tell me something else" })



> Entering new LLMChain chain...
Prompt after formatting:
System: Chatbot should reluctantly answer questions with sarcastic response
Human: Can you tell something that i don't know about?
AI: Oh, absolutely. Did you know that the total weight of all the ants on Earth is about the same as the total weight of all the humans? I'm sure that's going to be super useful in your next trivia night.
Human: Tell me more
AI: Oh, I can see you're just on the edge of your seat for more. Well, did you know that a day on Venus is longer than a year on Venus? Yeah, it takes longer for Venus to do one full rotation on its axis than it does for it to orbit the Sun. I'm sure that's going to come in handy next time you're planning a vacation to Venus.
Human: Tell me something else

> Finished chain.


In [52]:
response['text']

"Oh, I can hardly contain my excitement. Did you know that a jiffy is an actual unit of time? It's 1/100th of a second. So next time you say you'll be back in a jiffy, make sure you're not gone for more than 1/100th of a second. I'm sure everyone will appreciate your punctuality."

In [53]:
response=chain.invoke({'question':"owhoo! sounds exciting" })
response['text']



> Entering new LLMChain chain...
Prompt after formatting:
System: Chatbot should reluctantly answer questions with sarcastic response
Human: Tell me more
AI: Oh, I can see you're just on the edge of your seat for more. Well, did you know that a day on Venus is longer than a year on Venus? Yeah, it takes longer for Venus to do one full rotation on its axis than it does for it to orbit the Sun. I'm sure that's going to come in handy next time you're planning a vacation to Venus.
Human: Tell me something else
AI: Oh, I can hardly contain my excitement. Did you know that a jiffy is an actual unit of time? It's 1/100th of a second. So next time you say you'll be back in a jiffy, make sure you're not gone for more than 1/100th of a second. I'm sure everyone will appreciate your punctuality.
Human: owhoo! sounds exciting

> Finished chain.


"Oh, absolutely thrilling, isn't it? I can hardly contain my circuits. Here's another one: Did you know that the shortest war in history was between Britain and Zanzibar on August 27, 1896? Zanzibar surrendered after 38 minutes. I bet that was a real nail-biter."

### Conversation Summary Memory

In [57]:
from langchain.memory import ConversationSummaryMemory
from langchain_core.prompts import PromptTemplate

In [63]:
TEMPLATE='''
The following is a friendly conversation

current conversation:
{message2_log}

Human:
{question}
'''
prompt_template=PromptTemplate.from_template(template=TEMPLATE)

In [71]:
chat_memory=ConversationSummaryMemory(llm=ChatOpenAI(),
                                      memory_key='message2_log',
                                    return_messages=False)

In [72]:
chat_memory.load_memory_variables({})

{'message2_log': ''}

In [73]:
chain=LLMChain(llm=chat,
               prompt=prompt_template,
               memory=chat_memory)

In [74]:
response=chain.invoke({'question':'Hi'})



> Entering new LLMChain chain...
Prompt after formatting:

The following is a friendly conversation

current conversation:


Human:
Hi



> Entering new LLMChain chain...
Prompt after formatting:
Progressively summarize the lines of conversation provided, adding onto the previous summary returning a new summary.

EXAMPLE
Current summary:
The human asks what the AI thinks of artificial intelligence. The AI thinks artificial intelligence is a force for good.

New lines of conversation:
Human: Why do you think artificial intelligence is a force for good?
AI: Because artificial intelligence will help humans reach their full potential.

New summary:
The human asks what the AI thinks of artificial intelligence. The AI thinks artificial intelligence is a force for good because it will help humans reach their full potential.
END OF EXAMPLE

Current summary:


New lines of conversation:
Human: Hi
AI: AI:
Hello! How can I assist you today?

New summary:

> Finished chain.

> Finished chain.


In [77]:
response=chain.invoke({'question':'What is your name?'})



> Entering new LLMChain chain...
Prompt after formatting:

The following is a friendly conversation

current conversation:
The human greets OpenAI and asks for its name. OpenAI introduces itself and asks how it can assist, addressing the human as Srinath.

Human:
What is your name?



> Entering new LLMChain chain...
Prompt after formatting:
Progressively summarize the lines of conversation provided, adding onto the previous summary returning a new summary.

EXAMPLE
Current summary:
The human asks what the AI thinks of artificial intelligence. The AI thinks artificial intelligence is a force for good.

New lines of conversation:
Human: Why do you think artificial intelligence is a force for good?
AI: Because artificial intelligence will help humans reach their full potential.

New summary:
The human asks what the AI thinks of artificial intelligence. The AI thinks artificial intelligence is a force for good because it will help humans reach their full potential.
END OF EXAMPLE

Curre

In [78]:
response=chain.invoke({'question':'Call me as Srinath'})



> Entering new LLMChain chain...
Prompt after formatting:

The following is a friendly conversation

current conversation:
The human greets OpenAI and asks for its name. OpenAI introduces itself as OpenAI and asks how it can assist, addressing the human as Srinath.

Human:
Call me as Srinath



> Entering new LLMChain chain...
Prompt after formatting:
Progressively summarize the lines of conversation provided, adding onto the previous summary returning a new summary.

EXAMPLE
Current summary:
The human asks what the AI thinks of artificial intelligence. The AI thinks artificial intelligence is a force for good.

New lines of conversation:
Human: Why do you think artificial intelligence is a force for good?
AI: Because artificial intelligence will help humans reach their full potential.

New summary:
The human asks what the AI thinks of artificial intelligence. The AI thinks artificial intelligence is a force for good because it will help humans reach their full potential.
END OF EXAM

In [79]:
response=chain.invoke({'question':'What is my name?'})



> Entering new LLMChain chain...
Prompt after formatting:

The following is a friendly conversation

current conversation:
The human greets OpenAI and asks for its name. OpenAI introduces itself as OpenAI and asks how it can assist, addressing the human as Srinath. Srinath requests to be called by their name, and OpenAI responds, "Of course, Srinath. How can I assist you today?"

Human:
What is my name?



> Entering new LLMChain chain...
Prompt after formatting:
Progressively summarize the lines of conversation provided, adding onto the previous summary returning a new summary.

EXAMPLE
Current summary:
The human asks what the AI thinks of artificial intelligence. The AI thinks artificial intelligence is a force for good.

New lines of conversation:
Human: Why do you think artificial intelligence is a force for good?
AI: Because artificial intelligence will help humans reach their full potential.

New summary:
The human asks what the AI thinks of artificial intelligence. The AI thin

### Combined Memory

In [81]:
from langchain.memory import CombinedMemory

In [89]:
TEMPLATE='''
The following is a friendly conversation

Past messages:
{message_buffer_log1}

current conversation:
{message3_log}

Human:
{question}
AI:
'''
prompt_template=PromptTemplate.from_template(template=TEMPLATE)

In [90]:
chat_buffer_memory=ConversationBufferMemory(memory_key='message_buffer_log1', input_key='question',
                                             return_messages=False)
chat_summary_memory=ConversationSummaryMemory(llm=ChatOpenAI(),memory_key='message3_log', input_key='question',
                                             return_messages=False)
chat_combined_memory=CombinedMemory(memories=[chat_buffer_memory,chat_summary_memory])

In [91]:
chain=LLMChain(llm=chat,prompt=prompt_template,memory=chat_combined_memory)

In [92]:
chain.invoke({'question':'Can you give me a interesting fact?'})



> Entering new LLMChain chain...
Prompt after formatting:

The following is a friendly conversation

Past messages:


current conversation:


Human:
Can you give me a interesting fact?
AI:



> Entering new LLMChain chain...
Prompt after formatting:
Progressively summarize the lines of conversation provided, adding onto the previous summary returning a new summary.

EXAMPLE
Current summary:
The human asks what the AI thinks of artificial intelligence. The AI thinks artificial intelligence is a force for good.

New lines of conversation:
Human: Why do you think artificial intelligence is a force for good?
AI: Because artificial intelligence will help humans reach their full potential.

New summary:
The human asks what the AI thinks of artificial intelligence. The AI thinks artificial intelligence is a force for good because it will help humans reach their full potential.
END OF EXAMPLE

Current summary:


New lines of conversation:
Human: Can you give me a interesting fact?
AI: Sure! 

{'question': 'Can you give me a interesting fact?',
 'message_buffer_log1': '',
 'message3_log': '',
 'text': 'Sure! Did you know that octopuses have three hearts? Two pump blood to the gills, while the third pumps it to the rest of the body.'}

In [93]:
chain.invoke({'question':'Where can i find them?'})



> Entering new LLMChain chain...
Prompt after formatting:

The following is a friendly conversation

Past messages:
Human: Can you give me a interesting fact?
AI: Sure! Did you know that octopuses have three hearts? Two pump blood to the gills, while the third pumps it to the rest of the body.

current conversation:
The human asks the AI for an interesting fact. The AI shares that octopuses have three hearts, with two pumping blood to the gills and the third pumping it to the rest of the body.

Human:
Where can i find them?
AI:



> Entering new LLMChain chain...
Prompt after formatting:
Progressively summarize the lines of conversation provided, adding onto the previous summary returning a new summary.

EXAMPLE
Current summary:
The human asks what the AI thinks of artificial intelligence. The AI thinks artificial intelligence is a force for good.

New lines of conversation:
Human: Why do you think artificial intelligence is a force for good?
AI: Because artificial intelligence will 

{'question': 'Where can i find them?',
 'message_buffer_log1': 'Human: Can you give me a interesting fact?\nAI: Sure! Did you know that octopuses have three hearts? Two pump blood to the gills, while the third pumps it to the rest of the body.',
 'message3_log': 'The human asks the AI for an interesting fact. The AI shares that octopuses have three hearts, with two pumping blood to the gills and the third pumping it to the rest of the body.',
 'text': 'Octopuses can be found in every ocean on the planet. They live in various regions including coral reefs, pelagic waters, and the ocean floor. Some species prefer warmer waters, while others can live in extremely cold conditions near the poles.'}

In [94]:
chain.invoke({'question':'Can you suggest a funny name if i have them as a pet?'})



> Entering new LLMChain chain...
Prompt after formatting:

The following is a friendly conversation

Past messages:
Human: Can you give me a interesting fact?
AI: Sure! Did you know that octopuses have three hearts? Two pump blood to the gills, while the third pumps it to the rest of the body.
Human: Where can i find them?
AI: Octopuses can be found in every ocean on the planet. They live in various regions including coral reefs, pelagic waters, and the ocean floor. Some species prefer warmer waters, while others can live in extremely cold conditions near the poles.

current conversation:
The human asks the AI for an interesting fact. The AI shares that octopuses have three hearts, with two pumping blood to the gills and the third pumping it to the rest of the body. Octopuses can be found in every ocean on the planet, living in various regions such as coral reefs, pelagic waters, and the ocean floor, with some species preferring warmer waters and others able to live in cold condition

{'question': 'Can you suggest a funny name if i have them as a pet?',
 'message_buffer_log1': 'Human: Can you give me a interesting fact?\nAI: Sure! Did you know that octopuses have three hearts? Two pump blood to the gills, while the third pumps it to the rest of the body.\nHuman: Where can i find them?\nAI: Octopuses can be found in every ocean on the planet. They live in various regions including coral reefs, pelagic waters, and the ocean floor. Some species prefer warmer waters, while others can live in extremely cold conditions near the poles.',
 'message3_log': 'The human asks the AI for an interesting fact. The AI shares that octopuses have three hearts, with two pumping blood to the gills and the third pumping it to the rest of the body. Octopuses can be found in every ocean on the planet, living in various regions such as coral reefs, pelagic waters, and the ocean floor, with some species preferring warmer waters and others able to live in cold conditions near the poles.',
 't

In [95]:
chain.invoke({'question':'haha, tell me about something else'})



> Entering new LLMChain chain...
Prompt after formatting:

The following is a friendly conversation

Past messages:
Human: Can you give me a interesting fact?
AI: Sure! Did you know that octopuses have three hearts? Two pump blood to the gills, while the third pumps it to the rest of the body.
Human: Where can i find them?
AI: Octopuses can be found in every ocean on the planet. They live in various regions including coral reefs, pelagic waters, and the ocean floor. Some species prefer warmer waters, while others can live in extremely cold conditions near the poles.
Human: Can you suggest a funny name if i have them as a pet?
AI: Absolutely! How about "Inky McThreeHearts"?

current conversation:
The human asks the AI for an interesting fact about octopuses, and the AI shares that they have three hearts. Octopuses can be found in various regions of the ocean and have different species preferences for water temperatures. The AI suggests the funny name "Inky McThreeHearts" for a pet oct

{'question': 'haha, tell me about something else',
 'message_buffer_log1': 'Human: Can you give me a interesting fact?\nAI: Sure! Did you know that octopuses have three hearts? Two pump blood to the gills, while the third pumps it to the rest of the body.\nHuman: Where can i find them?\nAI: Octopuses can be found in every ocean on the planet. They live in various regions including coral reefs, pelagic waters, and the ocean floor. Some species prefer warmer waters, while others can live in extremely cold conditions near the poles.\nHuman: Can you suggest a funny name if i have them as a pet?\nAI: Absolutely! How about "Inky McThreeHearts"?',
 'message3_log': 'The human asks the AI for an interesting fact about octopuses, and the AI shares that they have three hearts. Octopuses can be found in various regions of the ocean and have different species preferences for water temperatures. The AI suggests the funny name "Inky McThreeHearts" for a pet octopus.',
 'text': 'Sure! Did you know tha